In [244]:
# DEV SCRIPT - DO NOT USE
import gen_funcs as gf
import ranked_funcs as rf
import clump_funcs as cf
import os
import numpy as np
import re
from scipy.linalg import norm

In [245]:
# Select desired figures
print("Running setup operations", flush=True)
selected_figs = ["old"]
# selected_figs = ["Fig2_Fig4", "Fig6", "Fig7", "Fig8"]

Running setup operations


In [246]:
# Load paths
path_paths = "paths.json"
dict_paths = gf.load_paths_all(path_paths)

(gen_funcs.load_paths_all)
	Reading paths


In [247]:
# Load data
path_data = dict_paths["path_data"]
paths_psliceout = gf.load_paths_psliceout(path_data, selected_figs)
data_all = gf.iteratively_load_data(paths_psliceout)

(gen_funcs.iteratively_load_data)
	Loading data
		Loading file ./data/old/psliceout1.dat


In [248]:
# Rank particles
path_ranked = dict_paths["path_ranked"]
n_neighbors = 100
fout_ranked = gf.iteratively_rank_neighbors(data_all, paths_psliceout, path_ranked, n_neighbors)

(gen_funcs.iteratively_rank_neighbors)
	Ranking particles
		Ranking file ./data/old/psliceout1.dat
(ranked_funcs.rank_neighbors)
	Checking if ranked file exists
		Ranked file ./output/ranked/ranked_100neigh_psliceout1.dat already exists. Skipping.


In [260]:
# Use ranked data to get COM coordinates
print("Using ranked data to get COM coordinates", flush=True)

# Calculate Roche density as threshold
G = 1
Omega_K = 1
H_g = 5.5
threshold_rho = cf.calculate_density_Roche(G, Omega_K, H_g)

# Get COM coordinates for all ranked files
# Setting up shell parameters
dr = 3e-4
n_shells = 5100
threshold_radius = 5.

# Do for all ranked files
data_rankedf = {}
coords_com_rankedf = {}
for folder in fout_ranked:
    for fin_ranked in fout_ranked[folder]:
        try:
            data_rankedf[fin_ranked]
        except KeyError:
            # Consolidate raw data with ranked data
            data_key = re.search("ranked_[0-9]+neigh_(.+).dat", fin_ranked).group(1)
            raw_coords, raw_idx = data_all[data_key]['raw_coords'], data_all[data_key]['raw_idx']

            # Get ranked data
            _data_ranked_red, _data_ranked = rf.read_ranked(fin_ranked, threshold_rho)

            # Get COM coordinates
            _coords_com_ranked, idxs_clump = rf.get_com_coords(_data_ranked_red, dr, n_shells, raw_coords, raw_idx, threshold_radius, var_shell=False)

            # Remove the final "clump" since it would have been empty
            _coords_com_ranked = {key: _coords_com_ranked[key][:-1] for key in _coords_com_ranked}

            # Save data
            data_rankedf[fin_ranked] = _data_ranked_red
            coords_com_rankedf[fin_ranked] = {"coords_com": _coords_com_ranked, "idxs_clump": idxs_clump}

Using ranked data to get COM coordinates


(ranked_funcs.read_ranked)
	Reading in and reducing ranked file ./output/ranked/ranked_100neigh_psliceout1.dat
(ranked_funcs.get_com_coords)
	Finding centres of mass of disconnected clumps (threshold radius:  5.0)

			COM of clump 0: [ 10.9048    -15.3215     -0.0517648]
			Number of particles in raw search space: 10000000
			Minimum distance to COM: 6.1767339686870505e-06
			Number of particles in reduced search space: 579168
			Number of particles in preclump: 448037.0
			Indices: [4286 4287 4288]
			Crossing point: 4286
			Number of particles in postclump: 443116.0
			Number of particles in ranked data at the end: 726032

			COM of clump 1: [24.6408   38.8009    0.127862]
			Number of particles in raw search space: 579168
			Minimum distance to COM: 50.849693828884185
			Number of particles in reduced search space: 0
			No more particles in search space. Exiting.


In [261]:
_coords_com_ranked

{'idx': [9767297],
 'radius': [4.65093e-07],
 'x': [10.9048],
 'y': [-15.3215],
 'z': [-0.0517648]}